<a href="https://colab.research.google.com/github/vanessaaleung/rawdata/blob/master/Copy_of_data_design_hw2_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [ ]:
import time # sleep
from urllib import (parse, request) # opening and reading and parsing URLs
import re # regex
from bs4 import BeautifulSoup as bs #scraping
from os import listdir # check dir

## Initialize params
Write modular code

In [ ]:
# Initialize params
SEARCH_TERM = 'samsung tv'
URL_BASE = 'https://www.ebay.com'
PATH = '/sch/i.html'
PARAM_SEARCH_TERM = '_nkw'
PARAM_PAGE_NUM = '_pgn'
START_PAGE = 1
END_PAGE = 10
FILENAME_PREFIX = 'ebay_samsung_tv_'
FILE_FORMAT = '.htm'

## Question a & b

Download the first 10 pages of search results and save as "ebay_samsung_tv_page_number.htm"

In [ ]:
def downloadPage(URL_BASE, PATH, 
                          PARAM_SEARCH_TERM, SEARCH_TERM, 
                          PARAM_PAGE_NUM, START_PAGE, END_PAGE,
                          FILENAME_PREFIX, FILE_FORMAT):
  # create a dict of query param names to values for building the query string: {'_nkw': 'samsung tv', '_pgn': 1}
  query_param_to_value = {}
  query_param_to_value[PARAM_SEARCH_TERM] = SEARCH_TERM
  query_param_to_value[PARAM_PAGE_NUM] = START_PAGE

  page_num = START_PAGE

  while page_num <= END_PAGE:
    query_param_to_value[PARAM_PAGE_NUM] = page_num # update page number in query string, then construct new URL
    url = URL_BASE + PATH + '?' + parse.urlencode(query_param_to_value) # url example: https://www.ebay.com/sch/i.html?_nkw=samsung+tv&_pgn=2
    output_file_path = (   # name of output file: ebay_samsung_tv_01.htm
        './{}'.format(
            FILENAME_PREFIX +
            str(page_num).zfill(2) +
            FILE_FORMAT)
    )  
    request.urlretrieve(url, output_file_path)  # download the webpage
    page_num += 1
    time.sleep(10)  # Each page request needs to be followed by at least a 10-second pause
  print(listdir()) # check for downloaded items

## Question c 
Find the sponsored items on each search result page and print their URL to the screen.

In [ ]:
def printSponsored(SEARCH_TERM, START_PAGE, END_PAGE):
  # create a regex pattern object to search for the "SPONSORED" subsequence
  c = re.compile('.*S.*P.*O.*N.*S.*O.*R.*E.*D.*')
  
  page_num = START_PAGE
  while page_num <= END_PAGE:
    print('Looking for sponsored search results for "{}" on page {} ...'.format(SEARCH_TERM, page_num))
   
    # find all items
    file_pointer = open('./ebay_samsung_tv_{}.htm'.format(str(page_num).zfill(2))) # open the htm file for a given page
    soup = bs(file_pointer) # parse with beautiful soup
    search_result_container = soup.find('div', {'class': 'srp-river-results'}) # narrow down results, there could be items on recently viewed item side bar
    tagblocks = set(search_result_container.find_all('div', {'class': 's-item__title--tagblock'})) # find all div's with classname 's-item__title--tagblock'

    for element in tagblocks:
      if (
          c.match(element.text) and # check if matching regex pattern
          element.parent.parent.name == 'a' and # and has an anchor element as a grandparent
          's-item__link' in element.parent.parent['class']
      ):
        print(element.parent.parent['href'])  # print link to screen
    page_num += 1

In [ ]:
# This is a another method we used to fetch urls of sponsored items. The function name is changed to printSponsoredMethod2.
def printSponsoredMethod2(SEARCH_TERM, START_PAGE, END_PAGE):
  # create a regex pattern object to search for the urls
  c = 'https.*?\?'
  
  page_num = START_PAGE
  while page_num <= END_PAGE:
    print('Looking for sponsored search results for "{}" on page {} ...'.format(SEARCH_TERM, page_num))
   
    # find all items
    file_pointer = open('./ebay_samsung_tv_{}.htm'.format(str(page_num).zfill(2)),encoding="utf-8") # open the htm file for a given page
    soup = bs(file_pointer) # parse with beautiful soup
    search_result_container = soup.find_all("a",attrs={"data-track":re.compile('.*UnifiedRankingScenario.*'),'class':'s-item__link'}) # only sponsored items' 'data track' attributes contain 'UnifiedRankingScenario'; class is defined so that review pages are not fetched
    item_urls=re.findall(c,str(search_result_container))# use regex to find urls
    for item_url in item_urls: # print the urls
        print(item_url)
    page_num += 1

## Code Execution

In [ ]:
downloadPage(URL_BASE, PATH, PARAM_SEARCH_TERM, SEARCH_TERM, PARAM_PAGE_NUM, START_PAGE, END_PAGE, FILENAME_PREFIX, FILE_FORMAT)

['.config', 'ebay_samsung_tv_05.htm', 'ebay_samsung_tv_06.htm', 'ebay_samsung_tv_10.htm', 'ebay_samsung_tv_09.htm', 'ebay_samsung_tv_01.htm', 'ebay_samsung_tv_02.htm', 'ebay_samsung_tv_07.htm', 'ebay_samsung_tv_08.htm', 'ebay_samsung_tv_03.htm', 'ebay_samsung_tv_04.htm', 'sample_data']


In [ ]:
printSponsored(SEARCH_TERM, START_PAGE, END_PAGE) # Fetch Sponsored Items using method 1

Looking for sponsored search results for "samsung tv" on page 1 ...
https://www.ebay.com/itm/Samsung-58-Class-4k-2160P-Smart-LED-TV-UN58MU6070EXZA/123941346695?epid=6032618554&_trkparms=ispr%3D1&hash=item1cdb7add87:g:oJMAAOSwfVFcXXPN&enc=AQAEAAACQBPxNw%2BVj6nta7CKEs3N0qUhSZnDKPf1auLiJz4QpUGIoYRTZKIDRmsR2TFZtennkz6ppNzlKbBIHkIEy1AXcMR0QQcJmdUVuoNooUd0MmDJmt6F8MxmZgubZgjB%2BoU63%2FoHh8hf8mqQdjPUXa%2BXnHmaW%2FZ89YKmDQ3uF1pI4d1LiMtotJk2lDj5ID8mfeNaJQJKb%2FBZ9isMOCQ3ZUQ8SDX8moFGrOANDh1tVk%2BnBKDfXEOqQXsZ5Fl993uDz2ygOB5rKyFKloHGEIkhk17e5KnL%2BbOz6hU7xv7OozAc81xXUeDAUlVtYw4Vpv00bIYGJ39cSOrJBnqSzpZAUWj3ok716rKUDiMUCeEnbxk7WsbMo25rI4KrTouyd%2FZB%2FwWO01o2ysmXAQgu5erTvbBQI0F8747Yzjm%2BWZLiJxGlIVaygxbZmGGfEVVe1lvYsqAeF13F07KSqaleYCg44AJHYbfKw2FlGoxSJ6Jqjbm0Zn%2FsQ4qGTRgFVfeXN9gjJ5z9qzpI84M9lKtqt6%2B0WjMnr%2FlyNUNjlUFY7LaquOMbZU8F8sYC1ZusVmmIojVcfEL3lXKeL4cnAiOF5bP4n4CPFvgj0tkRS%2F5%2Fk2uQNAX2rRkfjV7PtYSJ%2BcU4YFxrxIFHodG84vY197tYJUekWjbaU8egsADlUqygnm4Yi0QLVK4mxylc3GM0QRlCaNx%2BLVJqPjGCmdbY5UjQv2

In [ ]:
printSponsoredMethod2(SEARCH_TERM, START_PAGE, END_PAGE) # Fetch Sponsored Items using method 2

Looking for sponsored search results for "samsung tv" on page 1 ...
https://www.ebay.com/itm/Samsung-55-Class-2160-4K-UHD-LED-Smart-TV-UN55RU7300FXZA/123977682848?
https://www.ebay.com/itm/SAMSUNG-50-Class-4K-Ultra-HD-2160P-HDR-Smart-LED-TV-UN50RU7200-2019-Model/113998383444?
https://www.ebay.com/itm/Samsung-55-4K-Ultra-HD-HDR-Smart-QLED-TV-QN55Q60R/183806039372?
https://www.ebay.com/itm/Samsung-UN55RU7100-55-PurColor-Smart-4K-Ultra-HD-LED-TV-with-120-Motion-Rate/202638269096?
https://www.ebay.com/itm/Samsung-58-Class-4k-2160P-Smart-LED-TV-UN58MU6070EXZA/123941346695?
https://www.ebay.com/itm/Samsung-QN65LS03RAFXZA-Frame-65-QLED-4K-HD-Amazon-Alexa-Google-2019-QN65LS03R/283507587105?
https://www.ebay.com/itm/Samsung-QN65Q90RAFXZA-65-QLED-HDR-4K-UHD-Amazon-Alexa-Google-2019-QN65Q90R/273767288636?
https://www.ebay.com/itm/Samsung-QN49Q60RAFXZA-Flat-49-QLED-HDR-4K-HD-Amazon-Alexa-Google-2019-QN49Q60R/273920993470?
https://www.ebay.com/itm/Samsung-UN43RU7100-43-Smart-4K-Ultra-HD-TV-with-Goo